#  GAN

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
* https://www.kaggle.com/datasets/splcher/animefacedataset
* https://github.com/eriklindernoren/PyTorch-GAN

## Задачи для совместного разбора

1\. Обсудите основные шаги в обучении GAN.

In [1]:
import torch.nn as nn
import torch

gen = nn.Sequential(
    nn.ConvTranspose2d(100, 8, 4, 2, 1),
    nn.Tanh()
)

In [2]:
noise = torch.randn(16, 100, 1, 1)

In [ ]:
generated = gen(noise)
generated.shape

In [4]:
faked_labels = torch.zeros(16)

In [5]:
real = torch.randn(16, 8, 2, 2)
real_labels = torch.ones(16)

In [6]:
discriminator = nn.Sequential(
    #...
    nn.Flatten(start_dim=1),
    nn.LazyLinear(out_features=2)
)

In [ ]:
discriminator(generated).shape

In [ ]:
discriminator(real).shape

## Задачи для самостоятельного решения

<p class='task' id='1'></p>

1\. Создайте набор данных на основе датасета `animefacedataset`. Используя преобразования `torchvision`, приведите изображения к одному размеру и нормализуйте их. Выведите на экран несколько примеров изображений.

- [ ] Проверено на семинаре

In [9]:
import kagglehub
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, CenterCrop, Normalize, ToTensor
from torchvision.utils import make_grid

import matplotlib.pyplot as plt
import seaborn as sns

from torch.utils.data import DataLoader
import torch as th
import numpy as np

In [ ]:
path = kagglehub.dataset_download('splcher/animefacedataset')
print('Path to dataset files:', path)

In [11]:
image_size = 64
batch_size = 128
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

transform = Compose([
    Resize((64, 64)),
    CenterCrop((64, 64)),
    ToTensor(),
    Normalize(*stats)
])

In [ ]:
dataset = ImageFolder(path, transform=transform)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
dataset

In [13]:
def denorm(img_tensors):
    return img_tensors * stats[1][0] + stats[0][0]

def show_images(images, nmax=64):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(make_grid(denorm(images.detach()[:nmax]), nrow=8).permute(1, 2, 0))

def show_batch(dl, nmax=64):
    images, _ = next(iter(dl))
    show_images(images, nmax)

In [ ]:
show_batch(loader)

<p class='task' id='2'></p>

2\. Реализуйте архитектуру `DCGAN` и обучите модель. Подберите гиперпараметры таким образом, чтобы получаемые изображения стали достаточного качественными (четкими и без существенных дефектов). Во время обучения сохраняйте примеры генерации изображений из случайного шума и сравните, как менялось качество получаемых изображений в процессе обучения.

- [ ] Проверено на семинаре

In [15]:
from torch import optim, nn
from torchvision.utils import save_image
from IPython.display import display, Image as IPImage
from tqdm.notebook import tqdm
from PIL import Image
import imageio
import os

In [16]:
device = th.device('cuda' if th.cuda.is_available() else 'cpu')

In [17]:
nc = 3
nz = 128
ngf = 64
ndf = 64

In [18]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [19]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
generator = Generator().to(device)
generator.apply(weights_init)

In [21]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
discriminator = Discriminator().to(device)
discriminator.apply(weights_init)

In [ ]:
xb = torch.randn(batch_size, nz, 1, 1, device=device)
fake_images = generator(xb)
show_images(fake_images.cpu())

In [24]:
sample_dir = 'generated_images'
os.makedirs(sample_dir, exist_ok=True)

def save_samples(index, latent_tensors):
    fake_images = generator(latent_tensors)
    fake_fname = 'generated-images-{0:0=4d}.png'.format(index)
    
    save_image(denorm(fake_images), os.path.join(sample_dir, fake_fname), nrow=8)
    print('Saving', fake_fname)

In [25]:
start_idx = 1
epochs = 250
lr = 2e-4

fixed_latent = torch.randn(64, nz, 1, 1, device=device)

In [26]:
losses_g, losses_d = [], []
real_scores, fake_scores = [], []

opt_d = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
opt_g = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
criterion = nn.BCELoss()

In [ ]:
for epoch in range(epochs):
    for real_images, _ in tqdm(loader, desc=f'Epoch {epoch+1}/{epochs}'):
        
        discriminator.zero_grad()
        real_cpu = real_images.to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), 1, dtype=torch.float, device=device)
        output = discriminator(real_cpu).view(-1)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake = generator(noise)
        label.fill_(0)
        output = discriminator(fake.detach()).view(-1)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        opt_d.step()

        generator.zero_grad()
        label.fill_(1)
        output = discriminator(fake).view(-1)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        opt_g.step()
    
    print('Epoch [{}/{}], loss_g: {:.4f}, loss_d: {:.4f}'.format(epoch+1, epochs, errG.item(), errD.item()))

    save_samples(epoch + start_idx, fixed_latent)

In [28]:
torch.save(generator.state_dict(), 'generator_model.bin')
torch.save(discriminator.state_dict(), 'discriminator_model.bin')

In [29]:
def save_frames_as_video(filename, images_path):
    files = [os.path.join(sample_dir, f) for f in os.listdir(sample_dir) if images_path in f]
    files.sort()
    
    images = []
    for fname in files:
        img = Image.open(fname)
        images.append(img)
    
    imageio.mimsave(filename, images, duration=0.25, loop=0)

In [ ]:
save_frames_as_video('output.gif', 'generated')
display(IPImage(filename='./output.gif'))

In [ ]:
latent = torch.randn(batch_size, nz, 1, 1, device=device)
fake_images = generator(latent).cpu()
show_images(fake_images, nmax=2)

<p class='task' id='3'></p>

3\. Создайте наборы данных на основе архива `summer2winter_yosemite.zip`. Используя преобразования `torchvision`, приведите изображения к одному размеру и нормализуйте их. Выведите на экран несколько примеров изображений, расположив изображения из одной пары рядом по горизонтали.

- [ ] Проверено на семинаре

<p class='task' id='4'></p>

4\. Реализуйте архитектуру `CycleGAN` и обучите модель. Подберите гиперпараметры таким образом, чтобы получаемые изображения стали достаточного качественными (четкими и без существенных дефектов). Во время обучения сохраняйте примеры преобразования (в обе стороны) и  сравните, как менялось качество получаемых изображений в процессе обучения.

- [ ] Проверено на семинаре